# Text generation webui on colab
This is a merge of 2 repositories: [camenduru](https://github.com/camenduru/text-generation-webui-colab)'s one and [this](https://colab.research.google.com/drive/1QHfOpe0ClD48JORCdi5jrZjFyepNRXER#scrollTo=hKuocueuXnm5) (found in an issue page, I don't remember the author, sorry), and some tweaks from me.

This colab runs [text generation webui](https://github.com/oobabooga/text-generation-webui) by oobabooga so supports also gptq and ggml models, and is a way to test EdgeGPT without installing the webui, but being just a test tool probably I won't continue to develop it or fix it if there are small errors.

In [ ]:
#@markdown # Download dependencies
#@markdown Check first if runtime is set on gpu. The download is also very long, be patient.

%cd /content

!git clone https://github.com/oobabooga/text-generation-webui.git
%cd /content/text-generation-webui
!pip install -r requirements.txt
!pip install EdgeGPT

!mkdir /content/text-generation-webui/repositories
%cd /content/text-generation-webui/repositories
!git clone -b v1.2 https://github.com/camenduru/GPTQ-for-LLaMa.git
%cd GPTQ-for-LLaMa
!python setup_cuda.py install

%cd /content/text-generation-webui/extensions
!git clone https://github.com/GiusTex/EdgeGPT
%cd /content/text-generation-webui


/content
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 122545 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.15.0-1ubuntu0.2_amd64.deb ...
Unpacking libc-ares2:amd64 (1.15.0-1ubuntu0.2) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.35.0-1build1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.35.0-1build1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.35.0-1build1_amd64.deb ...
Unpacking aria2 (1.35.0-1build1) ...
Setting up libc-ares2:amd64 (1.15.0-1ubuntu0.2) ...
Setting up libaria2-0:amd64 (1.35.0-1build1) ...
Setting up aria2 (1.35.0-1build1) ...
Processing triggers for man-db (2.9.1-1) ...
Processing triggers for libc-bin (2.31-0ubuntu9.9) ...
Cloning into 'text-generation-webui'...
remote: Enumerating objects: 7995, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (76/76), done.
remote: Tota

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/text-generation-webui/repositories
Cloning into 'GPTQ-for-LLaMa'...
remote: Enumerating objects: 832, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 832 (delta 98), reused 92 (delta 92), pack-reused 717
Receiving objects: 100% (832/832), 485.94 KiB | 16.20 MiB/s, done.
Resolving deltas: 100% (502/502), done.
/content/text-generation-webui/repositories/GPTQ-for-LLaMa
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/set

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#@markdown # Starting the webui

#@markdown ### Download model

model = "wizard-vicuna-13B-GPTQ" #@param ["wizard-mega-13B-GGML", "pygmalion-7b-ggml-q5_1", "pygmalion-13b-4bit-128g", "wizard-vicuna-13B-GPTQ", "vicuna-13b-GPTQ-4bit-128g", "custom_model"] {allow-input: false}

#@markdown Your model isn't in the list? Choose "custom_model" from the above dropdown menù, then copy author, name, branch (usually is "main") from [huggingface](https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads) and choose model's folder name and paste them here:
author = "TheBloke" #@param {type:"string"}
model_name = "LLaMa-7B-GGML" #@param {type:"string"}
branch = "main" #@param {type:"string"}
folder_name = "TheBloke_LLaMa-7B-GGML" #@param {type:"string"}

is_a_4bit_128g_or_gptq_model = True #@param {type:"boolean"}
#@markdown If the model isn't recognised, try turning this on or off.

#@markdown  The models that have webui files to fix, like guanaco, mpt, etc. don't work.

# Data
models = {
    "wizard-mega-13B-GGML": ("TheBloke", "wizard-mega-13B-GGML", "main", "TheBloke_wizard-mega-13B-GGML"),
    "pygmalion-7b-ggml-q5_1": ("waifu-workshop", "pygmalion-7b-ggml-q5_1", "main", "waifu-workshop_pygmalion-7b-ggml-q5_1"),
    "pygmalion-13b-4bit-128g": ("notstoic", "pygmalion-13b-4bit-128g", "main", "notstoic_pygmalion-13b-4bit-128g"),
    "wizard-vicuna-13B-GPTQ": ("TheBloke", "wizard-vicuna-13B-GPTQ", "main", "TheBloke_wizard-vicuna-13B-GPTQ"),
    "vicuna-13b-GPTQ-4bit-128g": ("anon8231489123", "vicuna-13b-GPTQ-4bit-128g", "main", "anon8231489123_vicuna-13b-GPTQ-4bit-128g"),
    "custom_model": (author, model_name, branch, folder_name),
}

# Download the model (if it hasn't been downloaded already)
huggingface_org, huggingface_repo, huggingface_branch, model_name = models[model]
![[ ! -f models/$model_name/config.json ]] && python download-model.py $huggingface_org/$huggingface_repo --branch $huggingface_branch

print(f"\033[1;32;1m\n --> Keep waiting, you need something like Running on public URL: https://5fa8b5668f81ae6fde.gradio.live \n\033[0;37;0m")

# Starting the web UI
if is_a_4bit_128g_or_gptq_model:
  cmd = f"python server.py --share --chat --extensions EdgeGPT --model {model_name} --groupsize 128 --wbits 4 --settings settings-colab.json"
else:
  cmd = f"python server.py --share --chat --extensions EdgeGPT --model {model_name} --settings settings-colab.json"

print(cmd)
!$cmd


 --> Keep waiting, you need something like Running on public URL: https://5fa8b5668f81ae6fde.gradio.live 

python server.py --share --chat --extensions EdgeGPT --model TheBloke_wizard-vicuna-13B-GPTQ --groupsize 128 --wbits 4 --settings settings-colab.json
2023-05-31 19:26:46.607395: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
INFO:Loading TheBloke_wizard-vicuna-13B-GPTQ...
INFO:Found the following quantized model: models/TheBloke_wizard-vicuna-13B-GPTQ/wizard-vicuna-13B-GPTQ-4bit.compat.no-act-order.safetensors
INFO:Loaded the model in 48.61 seconds.

INFO:Loading the extension "EdgeGPT"...

Thanks for using the EdgeGPT extension! If you encounter any bug or you have some nice idea to add, write it on the issue page here: https://github.com/GiusTex/EdgeGPT/issues
INFO:Loading the extension "gallery"...
Running on local URL:  http://127.0.0.1:7860
Run

# Extras
Download other extensions

In [ ]:
#@markdown # Download other extensions

extension_link = "https://github.com/wsippel/bark_tts" #@param {type:"string"}
extension_name = "bark_tts" #@param {type:"string"}

%cd /content/text-generation-webui/extensions
!git clone {extension_link}.git
%cd /content/text-generation-webui/extensions/{extension_name}
!pip install -r requirements.txt

%cd /content/text-generation-webui


/content/text-generation-webui/extensions
Cloning into 'bark_tts'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 104 (delta 55), reused 76 (delta 29), pack-reused 0
Receiving objects: 100% (104/104), 35.56 KiB | 11.85 MiB/s, done.
Resolving deltas: 100% (55/55), done.
/content/text-generation-webui/extensions/bark_tts
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-install-1htqcsgo/suno-bark_5e76cc37c408407d97ebbc406a6dfaa9
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-install-1htqcsgo/suno-bark_5e76cc37c408407d97ebbc406a6dfaa9
  Resolved https://github.com/suno-ai/bark.git to commit f6f2db527b13c4a3e52ed6fbac587aadc3723eb6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installi